In this exercise, we are implementing a basic form of the ID3 algorithm in Python.

# Data

The provided data are all comma-separated, with metadata at the beginning.

It's easy to hand type the variable names for each dataset. However, as they are so well-structured, we programmatically extract the variable names from the metadata.

In [1]:
import numpy as np
import pandas as pd

cad_file = "../data/CAD.data"
fishing_file = "../data/fishing.data"
contact_lenses_file = "../data/contact-lenses.data"
caesarian_file = "../data/caesarian.data"

cad = pd.read_csv(cad_file, header=None, comment='#')
fishing = pd.read_csv(fishing_file, header=None, comment='#')
contact_lenses = pd.read_csv(contact_lenses_file, header=None, comment='#')
caesarian = pd.read_csv(caesarian_file, header=None, comment='#')

In [2]:
# extract the variable names from the metadata
import string

def get_colnames(file):
    with open(file) as f:
        lines = []
        for line in f:
            if line.startswith('#'):
                lines.append(line.rstrip().strip('#'))
        names = [line.translate(str.maketrans('', '', string.punctuation)).split()[0] for line in lines]
        return names[2:-2] + [names[-1]]

cad_colnames = get_colnames(cad_file)
fishing_colnames = get_colnames(fishing_file)
contact_lenses_colnames = get_colnames(contact_lenses_file)
caesarian_colnames = get_colnames(caesarian_file)

cad.columns = cad_colnames
fishing.columns = fishing_colnames
contact_lenses.columns = contact_lenses_colnames
caesarian.columns = caesarian_colnames

In [3]:
# practice using pickle, although there is no need for it here
import pickle

infile = "../data/decision_tree_data.pickle"

with open(infile,'wb') as f:
    pickle.dump(cad, f)
    pickle.dump(fishing, f)
    pickle.dump(contact_lenses, f)
    pickle.dump(caesarian, f)

with open(infile, 'rb') as f:
    cad1 = pickle.load(f)
    fishing1 = pickle.load(f)
    contact_lenses1 = pickle.load(f)
    caesarian1 = pickle.load(f)

# ID3 Implementation

To get the entropy of a set, we implement the following:

$$ Entropy(S) = -\sum \limits_{i=1}^k p_i log_2p_i $$

A problem that arises is when a class value does not appear in a set, where we have a probability of 0. $$ log_20 = \infty $$ However, to make it easy for our algorithm, we assume $$ log_20 = 0 $$

To get the information gain of a value a, we implement the following:

$$ Gain(S,a) = Entropy(S) - \sum \limits_{v=value(a)} {\frac{\lvert S_v \rvert}{\lvert S \rvert}} Entropy(S_v) $$

We make two simplifying assumptions specific to the four datasets provided for this exercise.

- Class variable always appears in the last column.
- Numerical variable is of type 'int64'.

The main data structure used is tree. It is not difficult to construct trees based on the built-in Node class in Python. Although, to be able to distinguish different nodes with the same name/tag, which is requisite for tree building, something like an id needs to be added as an attribute of the node.

Our task is to reinvent the wheel, but it makes sense to focus our attention on the relevant wheel. We choose to use Treelib, as it makes a number of things easier. Treelib provides a basic ascii graph, indexes nodes with unique ids, and can export .dot files for graphviz. With a few extra steps, it is not difficult to extract rules from the resulting trees.

In [4]:
def get_entropy(arr):
    # set count to 0 if class value does not exist
    arr = [0 if v is None else v for v in arr]
    # get cardinality
    probs = np.divide(arr, np.sum(arr))
    # get log2 of probabilities, assign 0 to log2(0)
    log2p = [0 if prob == 0 else np.log2(prob) for prob in probs]
    return -np.sum(np.multiply(probs, log2p))

For categorical variables, selecting the attribute with the most information gain is straightforward. For numerical variables, there is an extra step, where we pick the best split for the numerical variable first. To get the initial splits, sort by numerical variable first, then get the mean of values that can separate the classes as splits. For example, in a binary classification, if the number 20 corresponds to the positive class and the number 22 corresponds to the negative class, then we take 21 as a split.

In [5]:
from collections import OrderedDict
import operator
import copy

# helper function extracted from get_attribute_w_max_information_gain()
def calculate_information_gain(cardinality_dict, entropy_dict, class_entropy):
    # sort cardinalities by keys
    cardinalities = list(dict(OrderedDict(sorted(cardinality_dict.items()))).values())
    # sort entropies by keys
    entropies = list(dict(OrderedDict(sorted(entropy_dict.items()))).values())
    # calculate information gain
    information_gain = class_entropy - np.sum(np.multiply(cardinalities, entropies))
    return information_gain

def get_attribute_w_max_information_gain(df):
    # get counts for class values
    class_value_counts = df.iloc[:,-1:].value_counts().values
    # get class entropy
    class_entropy = get_entropy(class_value_counts)
    # get unique class values
    class_values = pd.unique(df.iloc[:,-1])

    # initialize empty dictionary of information gain
    information_gain_dict = {}
    # assumes class variable is in the last column
    # get predictor variables/attributes
    attributes = [x for x in df.columns[:-1]]

    # assume data types limited to 'object' (string) and 'int64'
    for attr in attributes:
        # when attribute is categorical
        if df[attr].dtype == 'object':
            # store value counts to a variable
            value_count = df[attr].value_counts()
            # sum up value counts
            value_count_sum = value_count.sum()
            # store cardinality to a dictionary
            cardinality_dict = (value_count/value_count_sum).to_dict()
            # initialize empty list for storing counts
            count_arr = []
            # initialize empty dictionary for storing entropies
            entropy_dict = {}
            # get unique values of an attribute
            var_values = df[attr].unique()
            # get attribute column index
            col_ind = df.columns.get_loc(attr)
            # get count of class values by attribute values
            attr_class_count = df.iloc[:,[col_ind,-1]].value_counts()

            # iterate through attribute values to get each of their entropies
            for var_value in var_values:
                for class_value in class_values:
                    count_arr.append(attr_class_count.get((var_value, class_value)))
                entropy_dict[var_value] = get_entropy(count_arr)
                count_arr = []

            # store attribute entropy to a dictionary
            information_gain_dict[attr] = calculate_information_gain(cardinality_dict, entropy_dict, class_entropy)

        # when attribute is of type 'int64'
        elif df[attr].dtype == 'int64':
            # get attribute column index
            col_ind = df.columns.get_loc(attr)
            # get new dataframe with only current attribute and class variable
            new_df = df.iloc[:,[col_ind,-1]]
            # sort new dataframe by attribute
            new_df_sorted = new_df.sort_values(by=[attr]).reset_index(drop=True)
            # initialize empty list for split values
            split_values = []

            # split at the mean value of adjacent attribute values of bordering class values
            for j in range(len(new_df_sorted) - 1):
                # if class value of an instance is different from that of the next instance
                if new_df_sorted.iloc[j, 1] != new_df_sorted.iloc[j + 1, 1]:
                    # split value is the mean of the corresponding attribute values
                    split_value = (new_df_sorted.iloc[j, 0] + new_df_sorted.iloc[j + 1, 0]) / 2
                    split_values.append(split_value)

            # initialize empty information gain dictionary for numeric attributes
            info_gain_dict_num = {}

            # select split value with highest information gain
            for value in split_values:
                # initialize empty dictionary for entropies of split values
                num_entropy_dict = {}

                # get a fresh copy of the new dataframe
                # shallow copy would keep the label
                new_df_sorted2 = copy.deepcopy(new_df_sorted)
                # get index of values smaller than split
                ind = new_df_sorted2[attr] < float(value)
                # get index of values greater than or equal to split
                ind1 = new_df_sorted2[attr] >= float(value)

                # labels
                label1 = '< ' + str(value)
                label2 = '>= ' + str(value)

                # assign labels
                new_df_sorted2.iloc[ind, 0] = label1
                new_df_sorted2.iloc[ind1, 0] = label2

                # get value counts of labels
                attr_class_count = new_df_sorted2.value_counts()

                count_arr = []

                # iterate through attribute values to get each of their entropies
                for label in (label1, label2):
                    for class_value in class_values:
                        count_arr.append(attr_class_count.get((label, class_value)))
                    num_entropy_dict[label] = get_entropy(count_arr)
                    count_arr = []

                # get the number of each value in the attribute
                value_count = new_df_sorted2[attr].value_counts()
                # denominator of cardinality
                value_count_sum = value_count.sum()
                # cardinality dictionary
                cardinality_dict = (value_count/value_count_sum).to_dict()
                # get information gain of each value
                info_gain_dict_num[value] = calculate_information_gain(cardinality_dict, num_entropy_dict, class_entropy)

            # key of the value with max information gain
            max_gain_key = max(info_gain_dict_num.items(), key=operator.itemgetter(1))[0]
            # save the above information gain to dictionary
            information_gain_dict[(df[attr].name + '<' + str(max_gain_key))] = info_gain_dict_num[max_gain_key]
            # information_gain_dict[attr] = info_gain_dict_num[max_gain_key]

    # get key of the attribute with max information gain
    max_key = max(information_gain_dict.items(), key=operator.itemgetter(1))[0]
    return max_key

We use a recursive procedure to generate the decision tree. The stop conditions are: all instances are of the same class, where that class is taken as a leaf, or, there is only one attribute left, where the majority class is chosen as a leaf. Complication arises in the second case when there is no majority but an equal split of classes. We could either randomly choose a class or make a choice on the basis of some principled reason. We choose to use the majority class of the parent. The limit of the current implementation is that it only tracks the majority class of the immediate parent. In a case where classes are equally split in the parent node, idxmax() returns the first instance. This only pushes the "equal split" worry one level up. To apply the majority rule beyond the immediate parent, one approach (not implemented here) is to store the class split information in the tree nodes. Then it becomes easier to get information of class split at any level.

In [6]:
from treelib import Tree
import uuid, re
from IPython.display import clear_output
import time

# a helper function for growing trees
def grow_tree(tree, branch, new_id, parent_id):
    # if at root, create root node
    if tree.size() == 0:
        tree.create_node(branch, new_id)
    # if not at root, create node with provided parent
    else:
        tree.create_node(branch, new_id, parent=parent_id)

# a helper function for displaying tree graph dynamically
def dynamic_tree(tree, msg='', delay=0.1):
    if msg == '':
        tree.show()
        time.sleep(delay)
        clear_output(wait=True)
    else:
        print(msg)
        time.sleep(delay)
        clear_output(wait=True)
        tree.show()
        time.sleep(delay)
        clear_output(wait=True)


def id3(df, tree, parent = None, parent_majority_class = None):
    # cover cases with empty input
    if df.size == 0:
        return "Input Data is empty."

    # get the number of classes
    class_values_length = len(pd.unique(df.iloc[:,-1]))

    # use an uuid as a node id
    branch_id = uuid.uuid4()

    # grow leaf if all instances are of the same class
    if class_values_length == 1:
        leaf_value = df.iloc[0,-1:].values[0]
        grow_tree(tree, leaf_value, branch_id, parent)
        # dynamic_tree(tree, 'All instances are in the same class.')

    # grow leaf with majority class label if only one feature variable left
    elif len(df.columns) == 1:
        # instances equally split classes
        if len(set(df.iloc[:,-1:].value_counts())) == 1:
            leaf_value = parent_majority_class
            grow_tree(tree, leaf_value, branch_id, parent)
        # non-equal splits
        else:
            leaf_value = df.iloc[:,-1:].value_counts().idxmax()[0]
            grow_tree(tree, leaf_value, branch_id, parent)
        # dynamic_tree(tree, 'No more variable. Choose majority class.')

    # not leaf, grow tree
    else:
        # store majority class of parent
        parent_majority_class = df.iloc[:,-1:].value_counts().idxmax()[0]
        # get feature with max information gain
        node_var = get_attribute_w_max_information_gain(df)

        var_name = None

        # the name returned from max information gain is different for categorical variables and numerical variables
        # separate processing for numerical variables is needed because the return value carry extra information about split value
        if node_var in df.columns.values:
            grow_tree(tree, node_var, branch_id, parent)
            # dynamic_tree(tree)
            is_num_var = False
            col_values = list(np.unique(df[node_var].values))
        else:
            is_num_var = True
            node_var_name_split = str.split(node_var, '<')
            var_name = node_var_name_split[0]
            # numerical split value
            var_num_value = node_var_name_split[1]
            grow_tree(tree, var_name, branch_id, parent)
            # dynamic_tree(tree)
            split1 = var_name + '<' + var_num_value
            split2 = var_name + '>=' + var_num_value
            col_values = [split1, split2]

        # new parent is current branch id
        new_parent = branch_id

        # numerical variables
        if is_num_var:
            # two way split
            new_branch_id1 = uuid.uuid4()
            new_branch_id2 = uuid.uuid4()
            tree.create_node(col_values[0], new_branch_id1, parent=new_parent)
            # dynamic_tree(tree)
            tree.create_node(col_values[1], new_branch_id2, parent=new_parent)
            # dynamic_tree(tree)
            name_value = re.split('<|>=', col_values[0])
            name = name_value[0]
            val = name_value[1]
            ind1 = df[name] < float(val)
            ind2 = df[name] >= float(val)
            df1 = df[ind1]
            df2 = df[ind2]

            # remove node if no instances
            if df1.size == 0:
                tree.remove_node(new_branch_id1)
                # dynamic_tree(tree)
            else:
                new_bid = uuid.uuid4()
                # grow a leaf if all instances are of the same class
                if len(np.unique(df1.iloc[:,-1:].values)) == 1:
                    leaf_value = df1.iloc[0,-1:].values[0]
                    grow_tree(tree, leaf_value, new_bid, new_branch_id1)
                    # dynamic_tree(tree)
                # grow a leaf with majority class if the other split is empty
                elif df2.size == 0:
                    leaf_value = df1.iloc[:,-1:].value_counts().idxmax()[0]
                    grow_tree(tree, leaf_value, new_bid, new_branch_id1)
                    # dynamic_tree(tree)
                # otherwise, call id3 on new df without the current variable
                else:
                    new_df = df1.drop(var_name, axis=1)
                    id3(new_df, tree, new_branch_id1, parent_majority_class)

            # apply the same process on df2
            if df2.size == 0:
                tree.remove_node(new_branch_id2)
                # dynamic_tree(tree)
            else:
                new_bid = uuid.uuid4()
                if len(np.unique(df2.iloc[:,-1:].values)) == 1:
                    leaf_value = df2.iloc[0,-1:].values[0]
                    grow_tree(tree, leaf_value, new_bid, new_branch_id2)
                    # dynamic_tree(tree)
                elif df1.size == 0:
                    # leaf_value = df2.iloc[:,-1:].mode().values[0][0]
                    leaf_value = df2.iloc[:,-1:].value_counts().idxmax()[0]
                    grow_tree(tree, leaf_value, new_bid, new_branch_id2)
                    # dynamic_tree(tree)
                else:
                    new_df = df2.drop(var_name, axis=1)
                    id3(new_df, tree, new_branch_id2, parent_majority_class)

        # categorical variables
        # this is much more straightforward
        # just create a new node for each variable value
        # then call id3 on the new df without the current variable
        else:
            for value in col_values:
                new_branch_id = uuid.uuid4()
                tree.create_node(value, new_branch_id, parent=new_parent)
                # dynamic_tree(tree)
                ind = df[node_var] == value
                df_v = df[ind].drop(node_var, axis=1)
                id3(df_v, tree, new_branch_id, parent_majority_class)
    return tree

In [7]:
# id3(cad1, Tree())

In [8]:
# get decision trees for each dataset
dtree_cad = id3(cad1, Tree())
dtree_fishing = id3(fishing1, Tree())
dtree_contact_lenses = id3(contact_lenses1, Tree())
dtree_caesarian = id3(caesarian1, Tree())

In [9]:
# dtree_cad.show()
# dtree_fishing.show()
# dtree_contact_lenses.show()
# dtree_caesarian.show()

# Experiments

## Pruning

From experiments, it seems prepruning by way of stopping when there is less that 5% of samples left is not reducing the complexity of the trees for the given datasets.

Postpruning can include many methods. Here we experiment with collapsing paring branches where the leaves are of the same class. For example, if there are two adjacent leaves (sharing the same grandparent) with the same class, we move the leaf node to where the grandparent node is.

In [10]:
from collections import Counter

def prune_tree(tree):
    # assume tree depth greater than 2 for simplicity
    if tree.depth() < 3:
        return

    # get all leaves
    leaves = tree.leaves()

    parent_ids = []

    # extract grandparent ids from leaves
    for i in range(len(leaves)):
        parent_ids.append(tree.get_node(tree.get_node(tree.leaves()[i].predecessor(tree.identifier)).predecessor(tree.identifier)).identifier)

    counter = Counter(parent_ids)
    # keep grandparent with more than one grandchild leaf
    grandparent_w_more_than_one_leaves = {k:v for k, v in counter.items() if v > 1}

    # check if the above set of grandparents has the same class in each
    # the idea is, even if we get two leaves with the same grandparent, there could be a third branch from the same grandparent, where the leaf is deeper
    # if that leaf share the same class, we can prune all three branches
    for gk in grandparent_w_more_than_one_leaves.keys():
        current_node_leaves = tree.leaves(gk)
        current_leaves_tags = [leaf.tag for leaf in current_node_leaves]
        if len(set(current_leaves_tags)) == 1:
            # get the parent of the grandparent
            parent_of_g = tree.get_node(gk).predecessor(tree.identifier)
            # move class leaf to where the grandparent is
            tree.move_node(current_node_leaves[0].identifier, parent_of_g)
            # remove the grandparent subtree
            tree.remove_node(gk)

For our datasets, the above postpruning method has effects on the cad and the caesarian datasets.

## Visualization

There are many ways trees can be visualized. One popular package is graphviz. For simplicity, we use pygraphviz to create a png image from the dot file generated from Treelib.

In [11]:
import pygraphviz as pgv

def draw_tree(tr, name):
    infile = '../outputs/' + name + '.dot'
    outfile = '../outputs/' + name + '.png'
    tr.to_graphviz(infile)
    g = pgv.AGraph(infile)
    g.layout(prog="dot")
    g.draw(outfile)

In [12]:
draw_tree(dtree_cad, 'cad')
draw_tree(dtree_fishing, 'fishing')
draw_tree(dtree_contact_lenses, 'contact_lenses')
draw_tree(dtree_caesarian, 'caesarian')

## Rule extraction

We also attempt to extract rules from decision trees. To achieve that, we make use of the path_to_leaves() property of a Treelib tree.

In [13]:
import itertools

# function for extracting rulls from Treelib trees
def get_if_then_rules(tr):
    if_then_rules = []
    # get path to leaves
    ptl = tr.paths_to_leaves()
    # extrat tags from ids
    for i in range(len(ptl)):
        temp_list = []
        for j in range(len(ptl[i])):
            temp_list.append(tr.get_node(ptl[i][j]).tag)
        if_then_rules.append(temp_list)

    # add = between varible name and value
    for s in range(len(if_then_rules)):
        if_then_rules[s] = [str(i)+'='+str(j) for i,j in itertools.zip_longest(if_then_rules[s][0::2], if_then_rules[s][1::2])]
        if_then_rules[s][-1] = if_then_rules[s][-1].split('=')[0]

    # add -> between sets of variable-value pairs
    if_then_rules_formatted = []
    for s in range(len(if_then_rules)):
        joined = ' -> '.join(if_then_rules[s])
        if_then_rules_formatted.append(joined)

    return if_then_rules_formatted

In [14]:
cad_rules = get_if_then_rules(dtree_cad)
fishing_rules = get_if_then_rules(dtree_fishing)
contact_lenses_rules = get_if_then_rules(dtree_contact_lenses)
caesarian_rules = get_if_then_rules(dtree_caesarian)

In [15]:
cad_rules

['Cholesterol=Borderline -> Gender=F -> No',
 'Cholesterol=Borderline -> Gender=M -> No',
 'Cholesterol=High -> Gender=F -> Yes',
 'Cholesterol=High -> Gender=M -> Yes',
 'Cholesterol=Normal -> No']

In [16]:
fishing_rules

['Sky=Cloudy -> Yes',
 'Sky=Rainy -> Air=Cool -> No',
 'Sky=Rainy -> Air=Warm -> Wind=Strong -> Yes',
 'Sky=Rainy -> Air=Warm -> Wind=Weak -> No',
 'Sky=Sunny -> Wind=Strong -> Yes',
 'Sky=Sunny -> Wind=Weak -> Water=Cold -> No',
 'Sky=Sunny -> Wind=Weak -> Water=Moderate -> Yes',
 'Sky=Sunny -> Wind=Weak -> Water=Warm -> No']

In [17]:
contact_lenses_rules

['tearrate=normal -> astigmatism=no -> age=pre-presbyopic -> soft',
 'tearrate=normal -> astigmatism=no -> age=presbyopic -> prescription=hypermetrope -> soft',
 'tearrate=normal -> astigmatism=no -> age=presbyopic -> prescription=myope -> none',
 'tearrate=normal -> astigmatism=no -> age=young -> soft',
 'tearrate=normal -> astigmatism=yes -> prescription=hypermetrope -> age=pre-presbyopic -> none',
 'tearrate=normal -> astigmatism=yes -> prescription=hypermetrope -> age=presbyopic -> none',
 'tearrate=normal -> astigmatism=yes -> prescription=hypermetrope -> age=young -> hard',
 'tearrate=normal -> astigmatism=yes -> prescription=myope -> hard',
 'tearrate=reduced -> none']

In [18]:
caesarian_rules

['Cardiac=abnormal -> BP=high -> Delivery=late -> Age=Age<34.5 -> no',
 'Cardiac=abnormal -> BP=high -> Delivery=late -> Age=Age>=34.5 -> yes',
 'Cardiac=abnormal -> BP=high -> Delivery=normal -> yes',
 'Cardiac=abnormal -> BP=high -> Delivery=premature -> yes',
 'Cardiac=abnormal -> BP=low -> yes',
 'Cardiac=abnormal -> BP=normal -> Age=Age<26.5 -> no',
 'Cardiac=abnormal -> BP=normal -> Age=Age>=26.5 -> Delivery=late -> no',
 'Cardiac=abnormal -> BP=normal -> Age=Age>=26.5 -> Delivery=normal -> yes',
 'Cardiac=normal -> Age=Age<22.0 -> BP=high -> yes',
 'Cardiac=normal -> Age=Age<22.0 -> BP=low -> yes',
 'Cardiac=normal -> Age=Age<22.0 -> BP=normal -> Delivery=normal -> yes',
 'Cardiac=normal -> Age=Age>=22.0 -> Delivery=late -> BP=high -> no',
 'Cardiac=normal -> Age=Age>=22.0 -> Delivery=late -> BP=low -> no',
 'Cardiac=normal -> Age=Age>=22.0 -> Delivery=late -> BP=normal -> no',
 'Cardiac=normal -> Age=Age>=22.0 -> Delivery=normal -> BP=high -> no',
 'Cardiac=normal -> Age=Age>=2

# Conclusion

As a note, using Treelib simplified many operations needed for this project. We may even use the data field to store some information for some form of backtracking.
